# Identifying Events

This script takes netCDF files with daily wind and RH measurements and identifies strong wind events, filters them by RH to find dry wind events, and aggregates those events into summary grids describing event statistics such as median duration, magnitude, etc.

In [ ]:
import xarray as xr
import os
import numpy as np
import pandas as pd
import scipy.signal
import geopandas as gpd

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Keep attributes by default, such as when performing mathematical operations on DataArrays
# https://github.com/pydata/xarray/pull/2482
xr.set_options(keep_attrs=True)

## Utility functions

In [ ]:
def set_no_data_around_indexes(arr, indexes):
    def get_inverse_mask(arr, unmasked_indexes):
        """Generate a boolean mask for a 1D array masked at all indexes except for the unmasked indexes.
        """
        mask = np.ones(arr.shape[0], dtype=bool)
        mask.put(unmasked_indexes, 0)
        return mask
    
    mask = get_inverse_mask(arr, indexes)
    output = arr.copy()
    output[mask] = np.nan
    return output

In [ ]:
def peak_magnitude_mask_duration_prominence(arr, **kwargs):
    peak_indexes = scipy.signal.find_peaks(arr, **kwargs)[0]
    
    peak_prominence = scipy.signal.peak_prominences(arr, peak_indexes)
    peak_widths = scipy.signal.peak_widths(arr, peak_indexes, prominence_data=peak_prominence)[0]
    
    presence_mask = arr.copy()
    presence_mask[peak_indexes] = 1
    
    widths = arr.copy()
    widths[peak_indexes] = peak_widths
    
    prominence = arr.copy()
    prominence[peak_indexes] = peak_prominence[0]
    
    prominence_out = set_no_data_around_indexes(prominence, peak_indexes)
    presence_out = set_no_data_around_indexes(presence_mask, peak_indexes)
    duration_out = set_no_data_around_indexes(widths, peak_indexes)
    magnitude_out = set_no_data_around_indexes(arr, peak_indexes)
    
    return magnitude_out, presence_out, duration_out, prominence_out

In [ ]:
def calculate_wind_event_stats(dataset, wind, rh, rh_threshold=36, wind_threshold=4, min_spacing=5, min_duration=1):
    """Take an xarray.Dataset and append variables describing wind event stats at event peaks.
    
    Parameters
    ----------
    dataset : xarray.Dataset
        The dataset to calculate events from.
    wind : str
        The name of the wind variable in the dataset.
    rh : str
        The name of the relative humidity variable in the dataset.
    rh_threshold : int
        The maximum RH to be considered an event.
    """
    output = dataset.copy()
    
    # Generate grids with event data at each peak
    magnitude, mask, duration, prominence = np.apply_along_axis(peak_magnitude_mask_duration_prominence, 0, output[wind].values, 
                                                            height=wind_threshold, distance=min_spacing, width=min_duration)
    
    output = output.assign({
        "magnitude": (["date", "y", "x"], magnitude),
        "mask": (["date", "y", "x"], mask),
        "duration": (["date", "y", "x"], duration),
        "prominence": (["date", "y", "x"], prominence),        
    })
    
    attrs = dataset.attrs

    # Mask all event data where the peak RH is below the threshold
    output["magnitude"] = output.magnitude.where(cond=output[rh] < rh_threshold, other=np.nan).assign_attrs(attrs)
    output["mask"] = output.mask.where(cond=output[rh] < rh_threshold, other=np.nan).assign_attrs(attrs)
    output["duration"] = output.duration.where(cond=output[rh] < rh_threshold, other=np.nan).assign_attrs(attrs)
    output["prominence"] = output.prominence.where(cond=output[rh] < rh_threshold, other=np.nan).assign_attrs(attrs)
    
    x = output.dims["x"]
    y = output.dims["y"]

    days = output.date.dt.dayofyear.values
    # Build an array the same shape as the output variables that just contains the day of the year
    day_arr = np.repeat(days, x*y).reshape((len(days), y, x))

    output = output.assign({
        "dayofyear": (("date", "y", "x"), day_arr)
    })

    # Mask using the pre-masked peaks
    output["dayofyear"] = output.dayofyear.where(cond=~xr.ufuncs.isnan(output.mask), other=np.nan).assign_attrs(attrs)
    
    return output

In [ ]:
def summarize_wind_event_stats(dataset):
    """Take an xarray.Dataset created by calculate_wind_event_stats and output a Dataset with wind event stats 
    summarized over time.
    """
    summary_arrs = [
        # Percent of all days within an event
        (dataset.mask.sum("date", keep_attrs=True) / dataset.dims["date"]) * 100,
        # Median duration
        dataset.duration.median("date", keep_attrs=True),
        # Cumulative duration
        dataset.duration.sum("date", keep_attrs=True),
        # Median magnitude
        dataset.magnitude.median("date", keep_attrs=True),
        # Max magnitude
        dataset.magnitude.max("date", keep_attrs=True),
        # Median day of year
        dataset.dayofyear.median("date", keep_attrs=True),
    ]

    summary_arr_names = [
        "percent_of_days",
        "duration_median",
        "duration_cumulative",
        "magnitude_median",
        "magnitude_max",
        "dayofyear_median",
    ]

    # When these are imported with rasterio, they get one attribute per day. Since we're switching from many days to just
    # one, we need to just have one attribute per.
    for arr in summary_arrs:
        arr.attrs["scales"] = np.array(arr.attrs["scales"][0])
        arr.attrs["offsets"] = np.array(arr.attrs["offsets"][0])

    summaries = {name:arr for arr, name in zip(summary_arrs, summary_arr_names)}

    # Combine summaries into 1 dataset
    summary = xr.Dataset(summaries)
    
    return summary

In [ ]:
def identify_regional_events(data, wind, rh, rolling_width=3, wind_threshold=4, min_spacing=3, min_duration=0, min_prominence=2, rh_threshold=36, min_date=121, max_date=305):
    """Take a Dataset that's been averaged over x and y to create a 1D timeline. Identify events and return a 
    dataframe of peaks and associated data.
    
    Parameters
    ----------
    data : xarray.Dataset
        The regionally averaged data with only a date dimension
    wind : str
        Name of the wind column to use.
    rh : str
        Name of the relative humididty column to use.
    min_date : int, default 121 (May 1)
        Earliest day of the year to consider a peak. Used to limit within the burning season.
    max_date : int, default 305 (Nov 1)
        Latest day of the year to consider a peak. Used to limit within the burning season.
    """
    data = data.copy().to_pandas().reset_index()

    # Calculate a rolling average of HDW to smooth events
    data[wind] = data[wind].rolling(window=rolling_width, center=True).mean()
    data[rh] = data[rh].rolling(window=rolling_width, center=True).mean()

    peak_indexes = scipy.signal.find_peaks(data[wind], 
                                           height=wind_threshold, 
                                           distance=min_spacing, 
                                           width=min_spacing, 
                                           prominence=min_prominence)[0]

    peaks = data.iloc[peak_indexes].copy()
    peak_widths = scipy.signal.peak_widths(data[wind], peak_indexes)
    peaks["width"] = peak_widths[0]

    # Create an index column that can be used in calculations
    peaks = peaks.reset_index()

    # Identify the number of days from the center of the event to the start (left) and end (right), rounded to nearest hour
    peaks["left_offset"] = pd.to_timedelta((peaks["index"] - peak_widths[2]).values, unit="day").round(freq="h")
    peaks["right_offset"] = pd.to_timedelta((peak_widths[3] - peaks["index"]).values, unit="day").round(freq="h")
    peaks["left"] = peaks.date - peaks.left_offset
    peaks["right"] = peaks.date + peaks.right_offset

    peaks = peaks[peaks[rh].lt(rh_threshold)]

    peaks = peaks[peaks.date.dt.dayofyear.gt(min_date) & peaks.date.dt.dayofyear.lt(max_date)]
    
    return peaks

In [ ]:
def plot_regional_peaks(data, peaks, wind, rolling_width=3, highlight_events=False):
    """Plot 1D wind data and associated peaks.
    
    Parameters
    ----------
    data : xarray.Dataset
    
    peaks : pandas.DataFrame
        Created by identify_regional_events.
    """
    data = data.copy().to_pandas().reset_index()

    # Calculate a rolling average of HDW to smooth events
    data[wind] = data[wind].rolling(window=rolling_width, center=True).mean()
    
    fig = px.line(data, x="date", y=wind, labels={wind: "Wind Velocity (m/s)", "date": ""})
    fig.add_trace(go.Scattergl(x=peaks.date, y=peaks[wind], mode="markers", name="Wind Peak"))
    
    if highlight_events:
        for _, peak in peaks.iterrows():
            fig.add_vrect(x0=peak.left, x1=peak.right, fillcolor="red", opacity=0.2, layer="below", line_width=0)
    
    return fig

In [ ]:
def above_percentile_mask(arr, p=0.95):
    """Take a 3D array of weather values over time and mask all values less than a given percentile.
    """
    threshold = np.nanquantile(arr, p, axis=[1, 2])
    
    x = arr.shape[2]
    y = arr.shape[1]

    # Build an array the same shape as the output variables that just contains the percentile threshold
    threshold_arr = np.repeat(threshold, x*y).reshape((len(arr), y, x))

    # Mask using the percentile threshold
    return np.where(arr > threshold_arr, 1, np.nan)

def below_percentile_mask(arr, p=0.05):
    """Take a 3D array of weather values over time and mask all values greater than a given percentile.
    """
    threshold = np.nanquantile(arr, p, axis=[1, 2])
    
    x = arr.shape[2]
    y = arr.shape[1]

    # Build an array the same shape as the output variables that just contains the percentile threshold
    threshold_arr = np.repeat(threshold, x*y).reshape((len(arr), y, x))

    # Mask using the percentile threshold
    return np.where(arr < threshold_arr, 1, np.nan)

## Constants

In [ ]:
data_dir = os.path.join("..", "data")

## AOI

In [ ]:
# Load Northwest Forest Plan boundary
nwfp_path = os.path.join(data_dir, "NWFP", "nwfpbnd.shp")
nwfp = gpd.read_file(nwfp_path)
nwfp.plot()

## gridMET Events

In [ ]:
gridmet = xr.open_dataset(os.path.join(data_dir, "gridMET", "gridMET.nc"))


gridmet = gridmet.drop("spatial_ref")

# Mask before calculating regional stats or percentiles to avoid including irrelevant areas
gridmet = gridmet.rio.clip(nwfp.geometry)

daily_medians_gridmet = gridmet[["vs", "rmin"]].median(["x", "y"])

### Pixel-wise events

Identify wind events and generate summaries of event statistics through time.

In [ ]:
# Just select summer months
gridmet = gridmet.sel(date=gridmet.date.dt.month.isin([5, 6, 7, 8, 9, 10]))

gridmet = calculate_wind_event_stats(gridmet, wind="vs", rh="rmin")
gridmet_summary = summarize_wind_event_stats(gridmet)

Export the summaries.

In [ ]:
gridmet_summary.rio.to_raster(os.path.join(data_dir, "summary_may-oct_gridMET.tif"))